In [1]:
import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")
#Pkg.add("PyCall")

In [2]:
using JuMP, GLPK
using DataFrames
using CSV
using PrettyTables
using Random

In [3]:
# Load the data
scenarios_df = CSV.read("../data/scenarios.csv", DataFrame)

n_scenarios = size(scenarios_df, 2)/3
n_scenarios = convert(Int, n_scenarios)

# create a dictonary with 200 dataframes for each scenario
all_scenarios = Dict()
for i in 1:n_scenarios
    df_helper = DataFrame(scenarios_df[:,3*i-2:3*i])
    df_helper[!,3] = df_helper[!,3] .* 1.0
    rename!(df_helper, [:"price", :"wind power", :"grid_excess"])
    all_scenarios[i] = df_helper
end

In [4]:
#W = 250
hours = 24

insample_scenarios = Dict()
out_of_sample_scenarios = Dict()
seeds = [123,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]
n_seeds = length(seeds)
Ws = [50, 150, 250, 500, 600, 750]

W_index = 0
for W in Ws
    
    for seed in seeds
        Random.seed!(seed)
        selected_scenarios = rand(1:n_scenarios, W)
        scenarios = Dict()
        counter = 1
        for i in selected_scenarios
            scenarios[counter] = all_scenarios[i]
            counter += 1
        end
        insample_scenarios[seed + n_seeds * W_index] = scenarios

        # create the list of index of scenarios out of sample
        out_of_sample = []
        for i in 1:n_scenarios
            if i ∉ selected_scenarios
                push!(out_of_sample, i)
            end
        end

        unselected_scenarios = Dict()
        counter = 1
        for i in out_of_sample
            unselected_scenarios[counter] = all_scenarios[i]
            counter += 1
        end
        out_of_sample_scenarios[seed + n_seeds * W_index] = unselected_scenarios
    end
    W_index += 1
end

alpha = 0.9
beta = 0.1
objectiv_values = []
exp_profits = []
profits_in = []
CVaR_values = []


Any[]

In [5]:
expected_profit_list = []
#create df with seeds as column and add the p_DA_values for each seed and definec the columns
bids_df = DataFrame()
W_index = 0

for W in Ws
    for seed in seeds
        scenarios = insample_scenarios[seed + n_seeds * (W_index)]

        # Create a new model with GLPK solver
        model = Model(GLPK.Optimizer)
        unregister(model, :p_DA)

        # Define the decision variables for hour
        @variable(model, p_DA[1:hours])
        @variable(model, delta[1:W,1:hours])
        #@variable(model, delta_up[1:W,1:hours])
        #@variable(model, delta_down[1:W,1:hours])
        @variable(model, zeta)
        @variable(model, eta[1:W] >= 0)

        # Define the objective function
        @objective(model, Max, (1-beta) * sum(1/W *
        (scenarios[i][hour,"price"] * p_DA[hour]
        + scenarios[i][hour,"price"] * delta[i, hour]*((0.9.*scenarios[i][hour,"grid_excess"]) + 1.2.*(1-scenarios[i][hour,"grid_excess"]))) for i in 1:W, hour in 1:hours)
        + beta  * (zeta - (1/(1-alpha)) * sum(1/W * eta[i] for i in 1:W)))

        # Define the constraints
        @constraint(model, [hour in 1:hours], p_DA[hour] <= 200)
        @constraint(model, [hour in 1:hours], p_DA[hour] >= 0)
        @constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == scenarios[i][hour,"wind power"] - p_DA[hour])
        #@constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == delta_up[i,hour] - delta_down[i,hour])
        #@constraint(model, [i in 1:W, hour in 1:hours], delta_down[i,hour] >= 0)
        #@constraint(model, [i in 1:W, hour in 1:hours], delta_down[i,hour] <= p_DA[hour])
        #@constraint(model, [i in 1:W, hour in 1:hours], delta_up[i,hour] >= 0)
        #@constraint(model, [i in 1:W, hour in 1:hours], delta_up[i,hour] + p_DA[hour] <= 200)

        @constraint(model, [i in 1:W], eta[i] >= 0)
        @constraint(model, [i in 1:W], -1 * sum((scenarios[i][hour,"price"] * p_DA[hour]
        + scenarios[i][hour,"price"] * delta[i, hour]*((0.9.*scenarios[i][hour,"grid_excess"]) + 1.2.*(1-scenarios[i][hour,"grid_excess"]))) for hour in 1:hours) + zeta - eta[i] <= 0)



        # Solve the optimization problem
        optimize!(model)

        # save the p_DA values
        p_DA_values = value.(p_DA)

        exp_profit = sum(1/W .*
        (scenarios[i][hour,"price"] * value.(p_DA[hour])
        + scenarios[i][hour,"price"] * value.(delta[i, hour])*((0.9.*scenarios[i][hour,"grid_excess"]) + 1.2.*(1-scenarios[i][hour,"grid_excess"]))) for i in 1:W, hour in 1:hours)


        #println(model)

        # Print the termination status
        status = termination_status(model)
        if status == MOI.OPTIMAL
            println("Expected profit ", exp_profit)
            println("p_DA: ", value.(p_DA))
            #add the p_DA values to the df
            bids_df[!, string(seed +  n_seeds * (W_index))] = value.(p_DA)
            push!(expected_profit_list, exp_profit)
            
        else
            println("No optimal solution found")
        end
    end
    W_index += 1
end

Expected profit 142566.52384515014
p_DA: [0.0, 0.0, 200.0, 200.0, 200.0, 0.0, 200.0, 0.0, 0.0, 200.0, 0.0, 0.0, 200.0, 0.0, 200.0, 0.0, 200.0, 200.0, 0.0, 200.0, 0.0, 83.1712850402327, 0.0, 200.0]
Expected profit 123951.06290457284
p_DA: [0.0, 0.0, 200.0, 200.0, 200.0, 0.0, 200.0, 0.0, 0.0, 200.0, 0.0, 0.0, 200.0, 0.0, 200.0, 0.0, 200.0, 200.0, 0.0, 200.0, 0.0, 0.0, 0.0, 200.0]
Expected profit 142566.52384515014
p_DA: [0.0, 0.0, 200.0, 200.0, 200.0, 0.0, 200.0, 0.0, 0.0, 200.0, 0.0, 0.0, 200.0, 0.0, 200.0, 0.0, 200.0, 200.0, 0.0, 200.0, 0.0, 83.1712850402327, 0.0, 200.0]
Expected profit 140492.2587612587
p_DA: [0.0, 0.0, 200.0, 200.0, -2.842170943040401e-14, 0.0, 200.0, 0.0, 0.0, 200.0, 0.0, 0.0, 200.0, 0.0, 200.0, 0.0, 200.0, 200.0, 0.0, 200.0, 0.0, 200.0, 0.0, 200.0]
Expected profit 141978.2366609292
p_DA: [0.0, 0.0, 200.0, 200.0, 200.0, 0.0, 200.0, 0.0, 0.0, 200.0, 0.0, 0.0, 200.0, 0.0, 200.0, 0.0, 200.0, 200.0, 0.0, 200.0, 0.0, 0.0, 0.0, 200.0]
Expected profit 149733.1571312961
p_D

In [6]:
hours = 24
profits_list = []
W_index=0

for W in Ws
    for seed in seeds
        unselected_scenarios = out_of_sample_scenarios[seed +  n_seeds * (W_index)]
        W_new = length(unselected_scenarios)
        p_DA_values = value.(bids_df[!, string(seed +  n_seeds * (W_index))])
        # calculate the expected profit for the out of sample scenarios
        exp_profit = 0
        for i in 1:W_new
            profit_day = 0
            for hour in 1:hours
                profit = 0
                profit += unselected_scenarios[i][hour,"price"] * p_DA_values[hour] # should the day ahead price be fixed to the first scenario? So instead of i, 1?
                profit += unselected_scenarios[i][hour,"price"] * (unselected_scenarios[i][hour,"wind power"]-p_DA_values[hour])*((0.9.*unselected_scenarios[i][hour,"grid_excess"]) + 1.2.*(1-unselected_scenarios[i][hour,"grid_excess"]))
                profit_day += profit
            end
            #push!(profits_out, profit_day)
            exp_profit += profit_day
        end
        exp_profit = exp_profit / W_new
        push!(profits_list, exp_profit)
    end
    W_index += 1
end

profits_list

144-element Vector{Any}:
 147443.01186803557
 151963.45063272826
 151062.90238415138
 151119.5966036024
 151069.81442383913
 150840.09878700957
 151384.8749010788
 150722.3164824095
 151678.5740836724
 150334.74255249012
      ⋮
 148151.3019356243
 148249.35838856167
 155219.1603898017
 150409.28178850317
 148964.80879616353
 151271.26495829967
 151811.78885458678
 150610.0499242844
 154907.50533144447

In [7]:
# create a dataframe with the expected profits, the seeds and the Ws
profits_df = DataFrame()

profits_df[!, "In sample scenarios"] = repeat(Ws, inner = length(seeds))
profits_df[!, "Seed"] = repeat(seeds, outer = length(Ws))
profits_df[!, "In sample profit"] = expected_profit_list
profits_df[!, "Out of sample profit"] = profits_list

# make a relative column
profits_df[!, "Relative profit"] = profits_df[!,"In sample profit"] ./ profits_df[!,"Out of sample profit"] .-1
profits_df

Row,In sample scenarios,Seed,In sample profit,Out of sample profit,Relative profit
,Int64,Int64,Any,Any,Float64
1,50,123,1.42567e5,147443.0,-0.0330737
2,50,2,1.23951e5,1.51963e5,-0.184336
3,50,3,1.42567e5,1.51063e5,-0.056244
4,50,4,1.40492e5,1.5112e5,-0.070324
5,50,5,1.41978e5,1.5107e5,-0.0601813
6,50,6,1.49733e5,1.5084e5,-0.00733851
7,50,7,1.40868e5,1.51385e5,-0.0694732
8,50,8,1.51279e5,1.50722e5,0.00369449
9,50,9,1.3031e5,1.51679e5,-0.14088


In [8]:
# save to a csv file 
CSV.write("1_5_results/relative_profits_one_price.csv", profits_df)


"1_5_results/relative_profits_one_price.csv"